# EAD0655 - Métodos Estatísticos de Projeção

### Integrantes:
- Marcelo Pesse - 5745229
- Caio Shimomura - 9811672
- Pedro Russel - 11189519
- Pedro Zaniolo - 10317109

# House Sales - Modelo de Regressão Linear

O projeto busca estabelecer um modelo preditivo, com o objetivo de prever o preço de uma casa em função de diversas caracteristicas da casa (Numero de quartos, número de banheiros, ano de construção, area construida, area total, etc).

Nossa variavel dependente sera a 'price' que representa o preço da casa e as demais variaveis serão indepentes.

In [ ]:
!pip install seaborn --upgrade

import numpy as np # Tratamento de dados numericos
import pandas as pd # Proessamento de dados, leitura de CSV, tratamento de tabela, etc
import matplotlib.pyplot as plt # Graficos basicos

import seaborn as sns # Graficos lindos

from scipy import stats

import sklearn #Biblioteca do scikit-learn, onde a magica da regressão acontece!
from sklearn import linear_model # Modelo de regressão linear
from sklearn.metrics import mean_squared_error, r2_score # Scores

from statsmodels.stats.outliers_influence import variance_inflation_factor #VIF
from yellowbrick.regressor import residuals_plot #Analise de residos

# Índice

- 1. Dataset
  - 1.1. Apresentação do dataset e variaveis
  - 1.2. Preparação da base de dados
- 2. Analise de correlação
- 3. Escolha de variaveis
  - 3.1. Histogramas
  - 3.2. Dispersão
- 4. Analise de multicolinearidade
  - 4.1. Correlação
  - 4.2. VIF (Variance inflation factor)
- 5. Modelo de regressão
  - 5.1. Primeiro modelo
    - 5.1.1. Dispersão Previsto X Real
    - 5.1.2. Analise dos resíduos
  - 5.2. Limitando o modelo a casas mais baratas que 1 milhão
  - 5.3. Reduzindo variaveis
  - 5.4. Transformada Logs
    - 5.4.1. Histogramas
    - 5.4.2. Dispersão
    - 5.4.3. Modelo
    - 5.4.4. Dispersão Previsto X Real
- 6. Conclussão
  - 6.1. TL;DR
  - 6.2. Conclussão & considerações finais

# 1. Dataset

A base de dados adotada para esse exercicio foi obtida no Kaggle, no link a seguir: https://www.kaggle.com/harlfoxem/housesalesprediction

A escolha foi feita com base na recomendação do autor como um bom dataset para regressões lineares simples e a popularidade do dataset na plataforma (medalha de ouro).

A base contem cerca de 21,6K datapoints sobre casas vendidas na região de King County, no estado de Whashington nos Estados Uunidos entre maio de 2014 e maio de 2015. Dado a concentração das informações somente em um ano (Pouco espaço de tempo para o mercado imobiliaria), iremos considerar o dataset como sendo cross-section.

Cada datapoint possui 21 variaveis, brevemente descritas a seguir:

In [ ]:
df_house = pd.read_csv('../input/housesalesprediction/kc_house_data.csv')

df_house

## 1.1. Variaveis

Segue as variaveis do dataset, bem como a escala de medida de cada uma e uma breve descrição da variavel.

| Nome da Variável | Escala de Medida | Descrição |
|---|---|---|
|id |Quali| ID unico da casa |
| date | Quanti | Data da venda |
| price | Quanti | Preço da venda |
| bedrooms | Quanti | Numero de quartos |
| bathrooms | Quanti | Numero de banheiros, onde .5 significa um banheiro sem chuveiro |
| sqft_living | Quanti | Metragem quadrada de area construida (Em pé quadrado) |
| sqft_lot | Quanti | Metragem quadrada de area total (Em pé quadrado) |
| floors | Quanti | Numero de andares |
| waterfront | Quanti | Variavel binaria, representa se a propriedade tem vista para o mar/agua |
| view | Quanti | Nota de 0 a 4 da vista da propriedade |
| condition | Quanti | Nota de 1 a 5 da condição da casa |
| grade | Quanti | Nota de 1 a 13, onde 1-3 é um nivel ruim de estrutura e design da casa, 7 é um nivel medio e 11-13 significa um alto nivel de estrutura e design |
| sqft_above | Quanti | Metragem quadrada de area construida acima do nivel da rua (Em pé quadrado) |
| sqft_basement | Quanti | Metragem quadrada de area construida abaixo do nivel da rua (Em pé quadrado) |
| yr_built | Quanti | Ano do inicio da construção da casa |
| yr_renovated | Quanti | Ano da ultima renovação feita |
| zipcode | Quali | Zipcode (Equivalente ao CEP brasileiro) |
| lat | Quanti | Latitude |
| long | Quanti | Longitude |
| sqft_living15 | Quanti | Metragem quadrada de area construida dos 15 vizinhos mais proximos (Em pé quadrado) |
| sqft_lot15 | Quanti | Metragem quadrada de area total dos 15 vizinhos mais proximos(Em pé quadrado) |

Segue tambem uma tabela com os valores da media, desvio-padrão, min e max de cada variavel.

In [ ]:
df_house.describe().drop(['count'])

## 1.2. Preparação da base de dados

Inicialmente examinou-se a base de dados para presença de datapoints faltantes.

In [ ]:
df_house.isna().sum()

Felizmente o dataset ja veio razoavelmente limpo do Kaggle e não possui nenhum valor nulo/faltante.

In [ ]:
df_house = df_house[(np.abs(stats.zscore( df_house[['price','bedrooms','bathrooms','sqft_living','sqft_lot','floors','view','condition','grade','sqft_above','sqft_basement','sqft_living15','sqft_lot15']] )) < 3).all(axis=1)]
df_house = df_house.drop(columns='waterfront')

Foi removido os outliears das variaveis que faziam sentido (Nao foi analisado o ZIP code por exemplo que é uma variavel qualitativa ou as informações de latitude e longitude).  
Para a remoção, foi utilizado o calculo do Z-score e a remoção de variaveis com z-score maior do que 3 ou menor do que -3.

Tambem foi removido a coluna 'waterfront' dado que com a remoção dos outliers, a coluna 'waterfront' ficou com somente 6 datapoints '1' o que não é muito estatisticamente significante.

# 2. Analise de correlação

Em seguida, foi feito uma analise de correlação entre as variaveis e a variavel 'price'.

In [ ]:
sns.set_theme(style="white")

corr = df_house.corr(method ='pearson')
mask = np.triu(np.ones_like(corr, dtype=bool))

f, ax = plt.subplots(figsize=(20, 20))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, annot = True, cbar_kws={"shrink": .5})

Para simplificar, tambem imprimimos só a correlação da variavel 'price' com as outras variaveis.

In [ ]:
print(corr['price'].sort_values(ascending=False))

# 3. Escolha de variaveis

Com base nas tabelas de correlação com a variavel 'price' escolhemos as seguintes variaveis:

| Variável | ρ |
|:---|:---|
| grade | 0.633291 |
| sqft_living | 0.622338 |
| sqft_living15 | 0.542608 |
| sqft_above | 0.527849 |
| bathrooms | 0.450734 |

Escolhemos as variaveis com **ρ** maior do que 0,5 e a variavel 'bathrooms' dado o valor ainda significamente alto de corr.

Descartamos a variavel 'lat' dado a falta de sentido na variavel. A correlação dessa variavel com a variavel 'price' só faz sentido com as limitações do dataset especifico. A correlação provelmente seria completamente diferente com um dataset diferente.

Talvez pudessemos criar uma nova variavel juntando as variaveis 'lat' e 'long', criando uma variavel 'distancia do centro da cidade', mas isso foge do escopo do trabalho.

In [ ]:
df_house = df_house[['price','grade','sqft_living','sqft_living15','sqft_above','bathrooms']]

## 3.1. Histogramas

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,15))
sns.histplot(df_house['price'], ax=axs[0][0])
sns.histplot(df_house['grade'], ax=axs[0][1])
sns.histplot(df_house['sqft_living'], ax=axs[0][2])
sns.histplot(df_house['sqft_living15'], ax=axs[1][0])
sns.histplot(df_house['sqft_above'], ax=axs[1][1])
sns.histplot(df_house['bathrooms'], ax=axs[1][2])

Analisando os graficos de histogramas das variaveis escolhidas, vemos que avariavel 'price', 'sqft_living', 'sqft_living15' e 'sqft_above' não possuem a desnidade de distribuição normal, tendo uma assimetria negativa (Com valores concentrados a esquerda).

Isso normalmente pode ser resolvido com transformadas das variaveis, mas faremos isso apos a geração de um modelo inicial.

## 3.2. Dispersão

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,15))
sns.scatterplot(data=df_house, x='price', y='price', ax=axs[0][0])
sns.scatterplot(data=df_house, x='price', y='grade', ax=axs[0][1])
sns.scatterplot(data=df_house, x='price', y='sqft_living', ax=axs[0][2])
sns.scatterplot(data=df_house, x='price', y='sqft_living15', ax=axs[1][0])
sns.scatterplot(data=df_house, x='price', y='sqft_above', ax=axs[1][1])
sns.scatterplot(data=df_house, x='price', y='bathrooms', ax=axs[1][2])

Analisando o grafico de dispersão das variaveis com relação a variavel preço, verificamos que temos um formato de cone, onde quanto maior o preço, mas temos dispersão dos valores (Ou o contrario).

Do ponto de vista dos dados e de nossa aplicação, isso faz sentido. Casas mais caras provavelmente levam em consideração mais variaveis (Qualidade da decoração, localidade, infraestruturas instaladas na propriedade, etc.). Enquanto casas mais baratas provavelmente so levam em consideração as variaveis analisadas.

Novamente, isso pode ser corrigido com uma transformada das variaveis.

# 4. Analise de Multicolinearidade

## 4.1. Correlação

In [ ]:
sns.set_theme(style="white")

corr = df_house.corr(method ='pearson')
mask = np.triu(np.ones_like(corr, dtype=bool))

f, ax = plt.subplots(figsize=(15, 15))
#cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, vmax=.3, center=0, square=True, linewidths=.5, annot = True, cbar_kws={"shrink": .5})

Infelizmente temos um problema forte de multicolinearidade nos nossos dados.

Nesse caso temos as seguintes opções:

* Excluir uma das variaveis
* Criar um score fatorial
* Criar um score somado/composto

**Entre a variavel 'sqft_living' e 'sqft_above' temos uma correlação de 0.85.**  
A correlação entre essas variaveis faz muito sentido dado a proximidade fisica dessas variaveis.  
Nesse sentido, faz sentido excluirmos uma dessas variaveis, vamos manter então a com melhor correlação com a variavel dependete. (No caso então manteremos a 'sqft_living').

**Entre a variavel 'sqft_living' e 'sqft_living15' temos uma correlação de 0.72.**  
A correlação entre essas variaveis faz sentido, dado que casas no mesmo bairro tendem a seguir o mesmo tamanho.  
Nesse caso especifico, uma criançao de uma nova variavel talvez seja interessante.  
Podemos criar um delta entre 'sqft_living' e 'sqft_living15', e talvez dividir por 'sqft_living15' conseguindo uma especie de *media* indicadora da diferença de tamanho da casa comparado com o resto do bairro.

In [ ]:
df_house['diff15'] = ( df_house['sqft_living'] - df_house['sqft_living15'] ) / df_house['sqft_living15']

print ('Max:', max(df_house['diff15']))
print ('Min:', min(df_house['diff15']))

print('')
corr = df_house.corr(method ='pearson')
print(corr['diff15'].sort_values(ascending=False))

Dado a pouca correlação com a variavel 'price', provavelmente não vale a pena manter essa nova variavel.

Vamos manter então somente as variaveis 'sqft_living', 'grade' e 'bathroom'.

## 4.2. VIF (Variance inflation factor)

In [ ]:
X = df_house[['sqft_living','grade','bathrooms']]

# VIF dataframe
vif_data = pd.DataFrame() 
vif_data['feature'] = X.columns 
  
# calculating VIF for each feature 
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))] 

print(vif_data)

Uhh, infelizmente temos valores de VIF maior do que cinco, o que indica uma alta multicolinearidade.

Mantendo so duplas de variaveis, infelizmente nunca chegamos em duas variaveis com VIF menor do que cinco, nesse caso, vamos manter as três variaveis escolhidas e ir removendo e medindo o R-quadrado do modelo para escolher o modelo que se aplica melhor.

# 5. Modelo de regressão

Finalmente iremos gerar o primeiro modelo de regressão linear!

## 5.1. Primeiro modelo

In [ ]:
reg = linear_model.LinearRegression()

X_train = df_house[['sqft_living','grade','bathrooms']]
y_train = df_house['price']
reg.fit(X_train, y_train)

y_pred = reg.predict(X_train)

print('X: sqft_living, grade e bathrooms\n')
print('Constante: ', reg.intercept_)
print('Coeficientes: ', reg.coef_)
print('R-quadrado: %.4f' % r2_score(y_train, y_pred))
print('R-quadrado Ajustado: %.4f' % (1 - (1-reg.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)) )

Utilizando as variaveis 'sqft_living', 'grade' e 'bathrooms', conseguimos um R-quadrado ajustado de 46,48%.

Nada mau para um modelo de regressão linear simples com 3 variaveis.

### 5.1.1. Dispersão Previsto X Real

In [ ]:
sns.set_theme(style="darkgrid")
f, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(x=y_train, y=y_pred)

x_plot = np.linspace(0, 1.2e6, 100)
y_plot = x_plot
plt.plot(x_plot, y_plot, color='r')
ax.set(xlabel='Real', ylabel='Previsto')
plt.show()

Identificamos uma tendencia do nosso modelo em prever erroneamente as casas com valores maiores.
Possivelmente se limitassemos nosso modelo a casas com preço menor do que 1.000.000 dolares, teremos resultados melhores.

Faz sentido casas tão caras assim dependerem de variaveis diferentes das que estamos usando (Basicamente tamanho da propriedade, uma nota de qualidade da propriedade e numero de banheiros).

### 5.1.2. Analise dos resíduos

In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
viz = residuals_plot(linear_model.LinearRegression(), X_train, y_train, is_fitted=True)

Felizmente, nossa analise dos residuos mostra que nossos erros são aparentemente randomicos, não verificamos nenhum padrão muito forte, o que indica que nosso modelo esta adequado.

Tambem podemos ver que pelo histograma nosso erro tem uma distribuição normal em torno do zero, o que tambem indica um bom modelo.

## 5.2. Limitando o modelo a casas mais baratas que 1 milhão

In [ ]:
reg = linear_model.LinearRegression()

df_house_cheap = df_house[df_house['price']<1e6]

X_train = df_house_cheap[['sqft_living','grade','bathrooms']]
y_train = df_house_cheap['price']
reg.fit(X_train, y_train)

y_pred = reg.predict(X_train)

print('X: sqft_living, grade e bathrooms\n')
print('Constante: ', reg.intercept_)
print('Coeficientes: ', reg.coef_)
print('R-quadrado: %.4f' % r2_score(y_train, y_pred))
print('R-quadrado Ajustado: %.4f' % (1 - (1-reg.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)) )

sns.set_theme(style="darkgrid")
f, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(x=y_train, y=y_pred)

x_plot = np.linspace(0, 1e6, 100)
y_plot = x_plot
plt.plot(x_plot, y_plot, color='r')
ax.set(xlabel='Real', ylabel='Previsto')
plt.show()

Infelizmente não vemos melhoras significativas com a remoção de casas mais caras do que 1 milhão, conseguimos um R-quadrado ajustado de 42,69%.

## 5.3. Reduzindo variaveis

In [ ]:
clf = linear_model.LinearRegression()

reg = linear_model.LinearRegression()

X_train = df_house[['sqft_living','grade']]
y_train = df_house['price']
reg.fit(X_train, y_train)

y_pred = reg.predict(X_train)

print('X: sqft_living, grade\n')

print('Constante: ', reg.intercept_)
print('Coefficients: ', reg.coef_)
print('R-quadrado: %.4f' % r2_score(y_train, y_pred))
print('R-quadrado Ajustado: %.4f' % (1 - (1-reg.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)) )

Removendo a variaveil 'bathroom' que possui o coeficiente de corr. mas baixo, conseguimos um R-quadrado ajustado de 46,20%, um pouco menor do que o nosso melhor resultado de 46,48%.

## 5.4. Transformada Logs

Vamos testar se a transformada log ajuda ou não a melhorar o nosso modelo!

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10,5))
sns.histplot(df_house['price'], ax=axs[0])
sns.histplot(np.log(df_house['price']), ax=axs[1])

Claramente a distribuição se aproxima mais de uma distribuição gaussiana apos uma transformada 'log'.

Provavelmente o mesmo pode ser inferido para as variaveis 'sqft_living', 'sqft_living15', 'sqft_above' que possuem distribuições muito similares a da variavel preço.

### 5.4.1. Histogramas

In [ ]:
df_house_log = df_house.copy()
df_house_log['price'] = np.log(df_house_log['price'])
df_house_log['sqft_living'] = np.log(df_house_log['sqft_living'])
df_house_log['sqft_living15'] = np.log(df_house_log['sqft_living15'])
df_house_log['sqft_above'] = np.log(df_house_log['sqft_above'])

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,15))
sns.histplot(df_house_log['price'], ax=axs[0][0])
sns.histplot(df_house_log['grade'], ax=axs[0][1])
sns.histplot(df_house_log['sqft_living'], ax=axs[0][2])
sns.histplot(df_house_log['sqft_living15'], ax=axs[1][0])
sns.histplot(df_house_log['sqft_above'], ax=axs[1][1])
sns.histplot(df_house_log['bathrooms'], ax=axs[1][2])

Apos a aplicação da transformada log, verificamos realmente que as curvas se aproximam mais de uma distribuição normal.

### 5.4.2. Dispersão

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,15))
sns.scatterplot(data=df_house_log, x='price', y='price', ax=axs[0][0])
sns.scatterplot(data=df_house_log, x='price', y='grade', ax=axs[0][1])
sns.scatterplot(data=df_house_log, x='price', y='sqft_living', ax=axs[0][2])
sns.scatterplot(data=df_house_log, x='price', y='sqft_living15', ax=axs[1][0])
sns.scatterplot(data=df_house_log, x='price', y='sqft_above', ax=axs[1][1])
sns.scatterplot(data=df_house_log, x='price', y='bathrooms', ax=axs[1][2])

Verificamos que a dispersão das variaveis fica com um formato mais proximo de um 'cilindro', diferentemente do cone que tinhamos anteriormente.

### 5.4.3. Modelo

In [ ]:
clf = linear_model.LinearRegression()

reg = linear_model.LinearRegression()

X_train = df_house_log[['sqft_living','grade','bathrooms']]
y_train = df_house_log['price']
reg.fit(X_train, y_train)

y_pred = reg.predict(X_train)

print('X log: sqft_living, grade e bathrooms\n')

print('Constante: ', reg.intercept_)
print('Coefficients: ', reg.coef_)
print('R-quadrado: %.4f' % r2_score(y_train, y_pred))
print('R-quadrado Ajustado: %.4f' % (1 - (1-reg.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)) )

Uhhh, a transformada Log não melhorou nosso modelo, nosso modelo mais simples tem um R-quadrado ajustado de 46,48% e nosso novo modelo utilizando a transformada log so conseguiu 45,61%.

Na duvida, faz sentido mantermos o modelo mais simples.

### 5.4.4. Dispersão Previsto X Real

In [ ]:
sns.set_theme(style="darkgrid")
f, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(x=y_train, y=y_pred)

x_plot = np.linspace(12, 14, 100)
y_plot = x_plot
plt.plot(x_plot, y_plot, color='r')
ax.set(xlabel='Real', ylabel='Previsto')
plt.show()

Nosso modelo de dispersão ficou menos disperso, infelizmente nosso R-quadrado mostra que realmente esse modelo não é melhor do que o modelo simples.

# 6. Conclussão



## 6.1. TL;DR

- Removemos outliears.
- Fizemos uma analise de correlação entre as variaveis e escolhemos as principais com maior valor de **p**.
- Testamos um modelo simples com 3 variaveis. **r-quadrado: 0.4648**
- Retiramos uma variavel para verificar se melhoraria o modelo. **r-quadrado: 0.4620**
- Testamos um modelo com transformada log. **r-quadrado: 0.4561**

## 6.2. Conclussão & considerações finais

Nosso modelo possui um R-quadrado abaixo do que gostariamos **(0.4648)**, porem ainda assim util.

Verificamos que a metragem de area construida, uma nota de avaliação de um corretor e numero de banheiros na casa são bons indicadores do preço de venda. Porem tambem chegamos a conclussão que casas mais caras possuem dispersão muito diferente de casas mais simples.
Provavelmente existem variaveis importantes que não conseguimos levar em consideração em nossa analise que influenciam essas caisas mais caras.

O modelo poderia ser utilizado para dar uma ideia de valor inicial em casas da região em um site da imobiliaria, mas provavelmente ainda não esta bom o suficiente para ser utilizado em uma aplicação comercial, como substituir ou auxiliar a analise feita pelo corretor.